In [ ]:
import pandas as pd
import numpy as np

# Load data

In [ ]:
def add_zeros(ags):
    ags = str(ags)
    length_ags = len(ags)
    if length_ags < 8:
        ags = "0" * (8-length_ags) + ags
    return ags

df_ags = pd.read_csv("../data/public/ags/master.csv")

# reading of old AGS List

# df_ags = pd.read_csv("ags_liste.csv", encoding="cp1252", engine="python", sep=";", dtype={"AGS": str})
# df_ags.rename(columns={"8-stellige AGS": "AGS", "Einheit": "Ort"}, inplace=True)
# df_ags.drop(
#     columns=["Zahl", "Insgesamt", "männlich", "weiblich", "Prüfzahl"], inplace=True
# )
# df_ags["AGS"] = df_ags["AGS"].apply(add_zeros)

df_anlagen = pd.read_csv(
    "Anlagenliste_AGS.csv", encoding="cp1252", engine="python", sep=";"
)
df_anlagen.rename(
    columns={
        "Standort der Anlage": "Ort",
        "AGS": "AGS_xls",
        "VET 2018 [t CO2 Äq]": "VET2018_t",
        "Haupttätigkeit nach TEHG": "TEHG_Nr",
        "Bezeichnung Haupttätigkeit nach TEHG": "TEHG_name",
    },
    inplace=True,
)
df_anlagen.drop(
    columns=[
        "Prüfsumme",
    ],
    inplace=True,
)


## Add industrie cateogires to facilities

In [ ]:
dict_industry_categories = {
    "miner_cement": ["Herstellung von Zementklinker"],
    "miner_chalk": ["Herstellung von Kalk"],
    "miner_glas": ["Herstellung von Glas"],
    "miner_ceram": ["Herstellung von Keramik"],
    "chem_basic": [
        "Herstellung von Salpetersäure",
        "Herstellung organischer Grundchemikalien",
        "Herstellung von Soda",
        "Herstellung von Adipinsäure",
        "Herstellung von Industrieruß",
    ],
    "chem_ammonia": ["Herstellung von Ammoniak"],
    "chem_other": ["Herstellung von Glyoxal und Glyoxylsäure"],
    "metal_steel": [
        "Herstellung von Roheisen und Stahl",
        "Verarbeitung von Eisenmetallen",
    ],
    "metal_steel_primary": [
        "Herstellung von Roheisen und Stahl"
    ],  # contained in metal_steel
    "metal_steel_secondary": [
        "Verarbeitung von Eisenmetallen"
    ],  # contained in metal_steel
    "metal_nonfe": [
        "Verarbeitung von Nichteisenmetallen",
        "Herstellung von Primäraluminium",
        "Verarbeitung von Metallerzen",
    ],
    "other_paper": ["Herstellung von Papier", "Herstellung von Zellstoff"],
    "other_food": [],
    "other_further": ["Herstellung von Mineralfasern", "Herstellung von Gips"],
}

# Herstellung von Mineralfasern ?, Herstellung von Gips ?


In [ ]:
df_anlagen["i_categorie"] = "No categorie"
for key, item in dict_industry_categories.items():
    if len(item) > 0:
        for name in item:
            df_anlagen.loc[df_anlagen["TEHG_name"] == name, "i_categorie"] = key

use only "Anlagen" from industrie

In [ ]:
df_anlagen = df_anlagen[df_anlagen["i_categorie"] != "No categorie"]

### remove duplicate AGS with identical Values in "Ort" (have to be added manually!)

In [ ]:
df_duplicates = df_ags[df_ags.duplicated(subset="description", keep=False)]
df_duplicates = df_duplicates.sort_values("description")

In [ ]:
df_ags_no_duplicates = df_ags.drop_duplicates(subset="description", keep=False)

# Find AGS for "Ort"

In [ ]:
df_anlagen.loc[:, "AGS"] = None
df_anlagen.loc[:, "Loc_matches"] = None
df_anlagen.loc[:, "match_type"] = None

# merge direct matches for column "Ort"
df_merge = df_anlagen.merge(df_ags_no_duplicates, left_on="Ort", right_on="description", how="left")
df_merge.loc[~df_merge["ags"].isna(), "match_type"] = "direct"

df_merge_open = df_merge[df_merge["ags"].isna()]

# match unique entries that 



# unique_ags_locs = df_ags["Ort"].unique()
# for i, row in df_anlagen.iterrows():
#     location = row["Ort"]
#     if row["Ort"] in unique_ags_locs:
#         for idx, ags_row in df_ags.iterrows():
#             ags_loc = ags_row["Ort"]
#             if location == ags_loc:
#                 df_anlagen.at[i, "AGS"] = ags_row["AGS"]
#                 df_anlagen.at[i, "match_type"] = "direct"
#     else:
#         print(location)

    

In [ ]:
df_merge["match_type"].value_counts(dropna=False)